# 🚀 Netconf interaction with Python ncclient

Welcome to the world of Model-based Network Automation with Python & Ncclient!</br>
This tutorial shows how to interact with a Cisco IOS-XR router using automated Netconf payloads and python.

🧰 Prerequisites

- 🐍 Python 3.9+
- 🔌 SSH access to your Cisco IOS-XR device. We are using the [Always-On Cisco IOSXR from DevNet Sandboxes](https://devnetsandbox.cisco.com/DevNet/catalog/ios-xr-always-on_ios-xr-always-on#instructions)
- 👤 `.env` file with its credentials and URL (available in this repository)

🔁 First of all, create a virtual environment with the following commands:

In [ ]:
!python3 -m venv .venv && source .venv/bin/activate
!pip install -r requirements.txt

✅ Let's import now our libraries and apply our environment variables from the `.env` file:

In [34]:
import re
import os
from ncclient import manager
from dotenv import load_dotenv
import xml.etree.ElementTree as ET

load_dotenv()

True

### 1️⃣ What can my device do? - aka. Which are its models

The very first step is to connect to our device, make sure that it has Netconf protocol enabled, and pull its capabilities. Later on, a specific model will be useful for us ...

In [ ]:
with manager.connect(
    host=os.getenv("XR_HOST"),
    port=830,
    username=os.getenv("XR_USER"),
    password=os.getenv("XR_PASS"),
    hostkey_verify=False,
    device_params={"name": "iosxr"},
    allow_agent=False,
    look_for_keys=False,
) as xr:

    for capability in xr.server_capabilities:
       print(capability)

📚 **There's a ton of models!** but we are looking for a specific one, as we want to setup a _loopback interface_.</br>
Let's filter it out! Using simple regex, we can look for whichever models that match the keyword `ifmgr`.</br>

> On IOS XR, configuration data for interfaces is managed by the Interface Manager (ifmgr) process. This is why this is our keyword

In [16]:
with manager.connect(
    host=os.getenv("XR_HOST"),
    port=830,
    username=os.getenv("XR_USER"),
    password=os.getenv("XR_PASS"),
    hostkey_verify=False,
    device_params={"name": "iosxr"},
    allow_agent=False,
    look_for_keys=False,
) as xr:

    for capability in xr.server_capabilities:
       model = re.search('module=([^&]*ifmgr[^&]*)&', capability)
       if model is not None:
           print(model.group(1))

Cisco-IOS-XR-ifmgr-cfg
Cisco-IOS-XR-ifmgr-oper


**Good!** The model that we need is ✨`Cisco-IOS-XR-ifmgr-cfg`✨.</br>
This is because we want to change the _configurations_ of the interfaces, not the state  - hence we go for the `cfg` file.</br>
Let's download it and have a look ...

In [17]:
with manager.connect(
    host=os.getenv("XR_HOST"),
    port=830,
    username=os.getenv("XR_USER"),
    password=os.getenv("XR_PASS"),
    hostkey_verify=False,
    device_params={"name": "iosxr"},
    allow_agent=False,
    look_for_keys=False,
) as xr:

    data_model_payload = xr.get_schema("Cisco-IOS-XR-ifmgr-cfg")
    with open("Cisco-IOS-XR-ifmgr-cfg.yang", "w") as f:
        f.write(data_model_payload.data)

🌳 Reading the raw YANG is a little bit overwhelming, isn't it?</br>
Let's render a tree using `pyang`

In [18]:
!pyang -f tree --tree-depth 5 Cisco-IOS-XR-ifmgr-cfg.yang

Cisco-IOS-XR-ifmgr-cfg.yang:12: error: module "Cisco-IOS-XR-types" not found in search path
Cisco-IOS-XR-ifmgr-cfg.yang:14: error: module "cisco-semver" not found in search path
module: Cisco-IOS-XR-ifmgr-cfg
  +--rw global-interface-configuration
  |  +--rw link-status?   Link-status-enum
  +--rw interface-configurations
     +--rw interface-configuration* [active interface-name]
        +--rw dampening
        |  +--rw args?                 enumeration
        |  +--rw half-life?            uint32
        |  +--rw reuse-threshold?      uint32
        |  +--rw suppress-threshold?   uint32
        |  +--rw suppress-time?        uint32
        |  +--rw restart-penalty?      uint32
        +--rw mtus
        |  +--rw mtu* [owner]
        |     +--rw owner    xr:Cisco-ios-xr-string
        |     +--rw mtu      uint32
        +--rw encapsulation
        |  +--rw encapsulation?         string
        |  +--rw capsulation-options?   uint32
        +--rw shutdown?                      empty
 

🔥 **Ah!** There are some errors in the rendering! This is because the elements of the tree are dependencies of other yang files.</br>
Let's download them all and try again ...

In [19]:
with manager.connect(
    host=os.getenv("XR_HOST"),
    port=830,
    username=os.getenv("XR_USER"),
    password=os.getenv("XR_PASS"),
    hostkey_verify=False,
    device_params={"name": "iosxr"},
    allow_agent=False,
    look_for_keys=False,
) as xr:

    data_model_payload = xr.get_schema("Cisco-IOS-XR-types")
    with open("Cisco-IOS-XR-types.yang", "w") as f:
        f.write(data_model_payload.data)

In [21]:
!pyang -f tree --tree-depth 5 Cisco-IOS-XR-ifmgr-cfg.yang

module: Cisco-IOS-XR-ifmgr-cfg
  +--rw global-interface-configuration
  |  +--rw link-status?   Link-status-enum
  +--rw interface-configurations
     +--rw interface-configuration* [active interface-name]
        +--rw dampening
        |  +--rw args?                 enumeration
        |  +--rw half-life?            uint32
        |  +--rw reuse-threshold?      uint32
        |  +--rw suppress-threshold?   uint32
        |  +--rw suppress-time?        uint32
        |  +--rw restart-penalty?      uint32
        +--rw mtus
        |  +--rw mtu* [owner]
        |     +--rw owner    xr:Cisco-ios-xr-string
        |     +--rw mtu      uint32
        +--rw encapsulation
        |  +--rw encapsulation?         string
        |  +--rw capsulation-options?   uint32
        +--rw shutdown?                      empty
        +--rw interface-virtual?             empty
        +--rw secondary-admin-state?         Secondary-admin-state-enum
        +--rw interface-mode-non-physical?   Interface-m

🧩 Using this yang tree, let's create a XML payload to **create a brand-new loopback interface**.</br></br>
Now, remember that **Netconf manages 3 different databases?**:

- `running-config`
- `startup-config`
- `candidate-config`

What we will do is the following:

🔧 Change `candidate-config` -> 🧪 Test the changes -> ⤵️ If OK, commit to `running-config`

In [25]:
config_xml = """
<config>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
    <interface-configuration>
      <interface-name>Loopback123</interface-name>
      <description>TESTDEMO</description>
      <active>act</active>
    </interface-configuration>
  </interface-configurations>
</config>
"""

In [ ]:
with manager.connect(
    host=os.getenv("XR_HOST"),
    port=830,
    username=os.getenv("XR_USER"),
    password=os.getenv("XR_PASS"),
    hostkey_verify=False,
    device_params={"name": "iosxr"},
    allow_agent=False,
    look_for_keys=False,
) as xr:

  test=xr.edit_config(config_xml, target='candidate', format='xml')
  print(test)

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:d829973c-5e59-4dfb-9992-60c6f706ffbd" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <ok/>
</rpc-reply>



✨ Looking good! Let's go ahead and **commit** this configuration. 

In [27]:
with manager.connect(
    host=os.getenv("XR_HOST"),
    port=830,
    username=os.getenv("XR_USER"),
    password=os.getenv("XR_PASS"),
    hostkey_verify=False,
    device_params={"name": "iosxr"},
    allow_agent=False,
    look_for_keys=False,
) as xr:

    test=xr.edit_config(config_xml, target='candidate', format='xml')
    if test.ok: 
        print(xr.commit())

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:55cadb94-07c2-4ddd-bead-46419c52297a" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <ok/>
</rpc-reply>



🔌 But what did I actually commit? Let's have a look at the **device's interfaces**.</br>
For this, we will get the configurations from the `running-config` and filter using a XML tag to only get the interfaces!

In [36]:
NS = {"ifcfg": "http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg"}
filter_xml = """
<filter>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
    <interface-configuration/>
  </interface-configurations>
</filter>
"""

with manager.connect(
    host=os.getenv("XR_HOST"),
    port=830,
    username=os.getenv("XR_USER"),
    password=os.getenv("XR_PASS"),
    hostkey_verify=False,
    device_params={"name": "iosxr"},
    allow_agent=False,
    look_for_keys=False,
) as xr:
    reply = xr.get_config(source="running", filter=filter_xml)
    root = ET.fromstring(reply.xml)
    configs = root.findall(".//ifcfg:interface-configuration", NS)

    interfaces = []
    for cfg in configs:
        name = cfg.findtext("ifcfg:interface-name", default="", namespaces=NS)
        desc = cfg.findtext("ifcfg:description", default="", namespaces=NS)
        active = cfg.findtext("ifcfg:active", default="", namespaces=NS)
        interfaces.append((name, desc, active))

    print(f"Found {len(interfaces)} interfaces in running-config:")
    for name, desc, active in sorted(interfaces):
        d = f' — "{desc}"' if desc else ""
        print(f"- [{active}] {name}{d}")

Found 37 interfaces in running-config:
- [act] Bundle-Ether1
- [act] Bundle-Ether2
- [act] GigabitEthernet0/0/0/0
- [act] GigabitEthernet0/0/0/1 — "test"
- [act] GigabitEthernet0/0/0/2 — "test"
- [act] GigabitEthernet0/0/0/3 — "test"
- [act] GigabitEthernet0/0/0/4
- [act] GigabitEthernet0/0/0/5
- [act] GigabitEthernet0/0/0/6
- [act] Loopback1
- [act] Loopback10
- [act] Loopback100 — "***TEST LOOPBACK****"
- [act] Loopback1000 — ""additional loopback interface added via automation""
- [act] Loopback1001
- [act] Loopback1002
- [act] Loopback11
- [act] Loopback12
- [act] Loopback123 — "TESTDEMO"
- [act] Loopback13
- [act] Loopback2
- [act] Loopback200
- [act] Loopback3
- [act] Loopback300
- [act] Loopback4
- [act] Loopback400
- [act] Loopback5
- [act] Loopback50
- [act] Loopback500
- [act] Loopback555 — "PRUEBA_KV"
- [act] Loopback6
- [act] Loopback600
- [act] Loopback7
- [act] Loopback700
- [act] Loopback8
- [act] Loopback9
- [act] Loopback999 — ""new interface added via automation""
- [

🔥 Last but not least, we want to **remove** the configurations that we just applied.</br>
It is as simple as tweaking a tiny bit our initial XML payload, and commit things again ...</br></br>
We will use the Netconf operation `delete`!

In [42]:
delete_config_xml = """
<config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
    <interface-configuration nc:operation="delete">
      <active>act</active>
      <interface-name>Loopback123</interface-name>
    </interface-configuration>
  </interface-configurations>
</config>
"""

with manager.connect(
    host=os.getenv("XR_HOST"),
    port=830,
    username=os.getenv("XR_USER"),
    password=os.getenv("XR_PASS"),
    hostkey_verify=False,
    device_params={"name": "iosxr"},
    allow_agent=False,
    look_for_keys=False,
) as xr:
    xr.edit_config(delete_config_xml, target='candidate', format='xml')
    print(xr.commit())

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:c5a698d2-2130-4e02-8304-08385d21358e" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <ok/>
</rpc-reply>



👀 Let's check if our interface is gone!

In [43]:
NS = {"ifcfg": "http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg"}
filter_xml = """
<filter>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
    <interface-configuration/>
  </interface-configurations>
</filter>
"""

with manager.connect(
    host=os.getenv("XR_HOST"),
    port=830,
    username=os.getenv("XR_USER"),
    password=os.getenv("XR_PASS"),
    hostkey_verify=False,
    device_params={"name": "iosxr"},
    allow_agent=False,
    look_for_keys=False,
) as xr:
    reply = xr.get_config(source="running", filter=filter_xml)
    root = ET.fromstring(reply.xml)
    configs = root.findall(".//ifcfg:interface-configuration", NS)

    interfaces = []
    for cfg in configs:
        name = cfg.findtext("ifcfg:interface-name", default="", namespaces=NS)
        desc = cfg.findtext("ifcfg:description", default="", namespaces=NS)
        active = cfg.findtext("ifcfg:active", default="", namespaces=NS)
        interfaces.append((name, desc, active))

    print(f"Found {len(interfaces)} interfaces in running-config:")
    for name, desc, active in sorted(interfaces):
        d = f' — "{desc}"' if desc else ""
        print(f"- [{active}] {name}{d}")

Found 38 interfaces in running-config:
- [act] Bundle-Ether1
- [act] Bundle-Ether2
- [act] GigabitEthernet0/0/0/0
- [act] GigabitEthernet0/0/0/1 — "test"
- [act] GigabitEthernet0/0/0/2 — "test"
- [act] GigabitEthernet0/0/0/3 — "test"
- [act] GigabitEthernet0/0/0/4
- [act] GigabitEthernet0/0/0/5
- [act] GigabitEthernet0/0/0/6
- [act] Loopback1
- [act] Loopback10
- [act] Loopback100 — "***TEST LOOPBACK****"
- [act] Loopback1000 — ""additional loopback interface added via automation""
- [act] Loopback1001
- [act] Loopback1002
- [act] Loopback1003
- [act] Loopback1004
- [act] Loopback11
- [act] Loopback12
- [act] Loopback13
- [act] Loopback2
- [act] Loopback200
- [act] Loopback3
- [act] Loopback300
- [act] Loopback4
- [act] Loopback400
- [act] Loopback5
- [act] Loopback50
- [act] Loopback500
- [act] Loopback555 — "PRUEBA_KV"
- [act] Loopback6
- [act] Loopback600
- [act] Loopback7
- [act] Loopback700
- [act] Loopback8
- [act] Loopback9
- [act] Loopback999 — ""new interface added via automat